<a href="https://colab.research.google.com/github/SkyTong/DS440/blob/main/LSTM_SOTA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Dependancies**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import *
from keras.callbacks import *
import os
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
#from commons import mean_absolute_percentage_error
from keras.layers import *
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.losses import LogCosh

# **Loading Data**

In [ ]:
PATH_TO_DATA = "/content/reg_interval20-23nd.csv"
data = pd.read_csv(PATH_TO_DATA)

In [ ]:
data.head()

,fee_to_rewardUSD,mediantransactionvalue30trxUSD,mining_profitability90trx,price30varUSD,price3momUSD,price3varUSD,price90momUSD,price90varUSD,sentinusdUSD,top100cap3trx,transactionvalueUSD,priceUSD
0,0.457,-0.240,-0.307,28623.0,-231.566,3878.0,-1017.000,573722.0,2.022007e+09,0.003,12757.0,7127.0
1,0.776,-0.261,-0.308,29355.0,-261.045,4729.0,-1046.000,581575.0,3.495276e+09,-0.020,16735.0,7250.0
2,0.770,-0.279,-0.310,29328.0,-42.402,3051.0,-878.841,587195.0,5.664998e+09,-0.020,23302.0,7390.0
3,0.583,-0.300,-0.311,29574.0,152.469,11526.0,-626.347,590895.0,3.470998e+09,-0.010,16928.0,7503.0
4,0.580,-0.325,-0.312,30437.0,376.272,10731.0,-565.003,593111.0,2.540099e+09,0.003,14695.0,7588.0


In [ ]:
data.shape

(1097, 12)

In [ ]:
y = data['priceUSD']

In [ ]:
X = data

In [ ]:
X['priceUSD'] = X['priceUSD'].shift(1,fill_value=12255)

In [ ]:
X

,fee_to_rewardUSD,mediantransactionvalue30trxUSD,mining_profitability90trx,price30varUSD,price3momUSD,price3varUSD,price90momUSD,price90varUSD,sentinusdUSD,top100cap3trx,transactionvalueUSD,priceUSD
0,0.457,-0.240,-0.307,28623.0,-231.566,3878.000,-1017.000,573722.0,2.022007e+09,0.003,12757.0,12255.0
1,0.776,-0.261,-0.308,29355.0,-261.045,4729.000,-1046.000,581575.0,3.495276e+09,-0.020,16735.0,7127.0
2,0.770,-0.279,-0.310,29328.0,-42.402,3051.000,-878.841,587195.0,5.664998e+09,-0.020,23302.0,7250.0
3,0.583,-0.300,-0.311,29574.0,152.469,11526.000,-626.347,590895.0,3.470998e+09,-0.010,16928.0,7390.0
4,0.580,-0.325,-0.312,30437.0,376.272,10731.000,-565.003,593111.0,2.540099e+09,0.003,14695.0,7503.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1092,1.932,-0.085,-0.406,82135.0,-179.890,8243.000,-2758.000,2474152.0,5.572254e+09,0.031,53190.0,16640.0
1093,2.283,-0.087,-0.405,76543.0,-258.919,8530.000,-2936.000,2479394.0,6.313827e+09,0.047,54195.0,16591.0
1094,2.161,-0.090,-0.404,82274.0,-257.127,1350.000,-2778.000,2489811.0,7.554070e+09,0.027,50405.0,16550.0
1095,1.235,-0.094,-0.403,86919.0,-69.693,287.999,-2661.000,2500289.0,4.493923e+09,0.030,42790.0,16570.0


In [ ]:
#X1 = data
#X1['target'] = X1['priceUSD'].shift(-1)


In [ ]:
#X1.dropna(subset=['target'], inplace=True)

In [ ]:
#X = data.iloc[:,:-1]

In [ ]:
#y=data.iloc[:,-1:]

In [ ]:
#y = data['priceUSD'].shift(-1,fill_value=1)

In [ ]:
#y = data['priceUSD']

In [ ]:
y

0        7127.0
1        7250.0
2        7390.0
3        7503.0
4        7588.0
         ...   
1092    16591.0
1093    16550.0
1094    16570.0
1095    16559.0
1096    16691.0
Name: priceUSD, Length: 1097, dtype: float64

In [ ]:
def create_dataset(dataset, time_step=1):
    dataX = []
    datay = []
    for i in range(len(dataset) - time_step):
        a = dataset.iloc[i:(i + time_step), :].values  # Use .iloc for DataFrame
        dataX.append(a)
        datay.append(dataset.iloc[i + time_step, -1])
    return np.array(dataX), np.array(datay)


In [ ]:
#time_step = 15
#X2 ,y2 = create_dataset(X1, time_step)

#print("X: ", X2.shape)
#print("y: ", y2.shape)

In [ ]:
#X2

In [ ]:
#y2

In [ ]:
#y2_reshaped = y2.reshape(-1, 1, 1)

In [ ]:
#y2_reshaped

In [ ]:
X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.2, train_size=0.8, shuffle=False, random_state=8)

In [ ]:
X_train.shape

(877, 12)

In [ ]:
estimators=[]

In [ ]:
estimators.append(['robust',RobustScaler()])

In [ ]:
estimators.append(['mixmax',MinMaxScaler()])

In [ ]:
scale=Pipeline(estimators,verbose=True)

In [ ]:
X_train=scale.fit_transform(X_train)

[Pipeline] ............ (step 1 of 2) Processing robust, total=   0.0s
[Pipeline] ............ (step 2 of 2) Processing mixmax, total=   0.0s


In [ ]:
X_test=scale.transform(X_test)

In [ ]:
X_train=np.reshape(X_train,(X_train.shape[0],1,X_train.shape[1]))

In [ ]:
X_test=np.reshape(X_test,(X_test.shape[0],1,X_test.shape[1]))

In [ ]:
y_train=y_train.values

In [ ]:
y_train=np.reshape(y_train, (y_train.shape[0],1,1))

In [ ]:
y_test=y_test.values

In [ ]:
y_test=np.reshape(y_test,(y_test.shape[0],1,1))

In [ ]:
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')

In [ ]:
X_train.shape

(877, 1, 12)

In [ ]:
X_train

array([[[0.        , 0.1867065 , 0.03035714, ..., 0.40980393,
         0.        , 0.1159221 ]],

       [[0.01123754, 0.17102315, 0.02946429, ..., 0.39477125,
         0.00163186, 0.0339292 ]],

       [[0.01102617, 0.15758029, 0.02767857, ..., 0.39477125,
         0.00432579, 0.03589588]],

       ...,

       [[0.05830133, 0.44660193, 0.03214286, ..., 0.4130719 ,
         0.1255354 , 0.38865083]],

       [[0.0585127 , 0.44436148, 0.03035714, ..., 0.43333334,
         0.07643056, 0.39578202]],

       [[0.05301723, 0.44286782, 0.02857143, ..., 0.43986928,
         0.10168917, 0.39096928]]], dtype=float32)

In [ ]:
y_train.shape

(877, 1, 1)

In [ ]:
y_train

array([[[ 7127.]],

       [[ 7250.]],

       [[ 7390.]],

       [[ 7503.]],

       [[ 7588.]],

       [[ 7953.]],

       [[ 8273.]],

       [[ 7943.]],

       [[ 7933.]],

       [[ 8120.]],

       [[ 8146.]],

       [[ 8151.]],

       [[ 8605.]],

       [[ 8778.]],

       [[ 8693.]],

       [[ 8869.]],

       [[ 8907.]],

       [[ 8859.]],

       [[ 8670.]],

       [[ 8672.]],

       [[ 8693.]],

       [[ 8469.]],

       [[ 8410.]],

       [[ 8349.]],

       [[ 8466.]],

       [[ 8744.]],

       [[ 9049.]],

       [[ 9349.]],

       [[ 9394.]],

       [[ 9366.]],

       [[ 9393.]],

       [[ 9398.]],

       [[ 9356.]],

       [[ 9233.]],

       [[ 9421.]],

       [[ 9706.]],

       [[ 9798.]],

       [[ 9823.]],

       [[10083.]],

       [[ 9932.]],

       [[ 9968.]],

       [[10331.]],

       [[10289.]],

       [[10272.]],

       [[10152.]],

       [[ 9916.]],

       [[ 9723.]],

       [[ 9840.]],

       [[10091.]],

       [[ 9612.]],



In [ ]:
#X_train, X_test, y_train, y_test =train_test_split(X2,y2_reshaped, test_size=0.2, train_size=0.8, shuffle=False, random_state=8)

In [ ]:
#X_train.shape[2]

In [ ]:
#y_train.shape

In [ ]:
#X_test.shape

# **Model Architecture + Training**

In [ ]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [ ]:

adam = optimizers.Adam(learning_rate=lr_schedule(0), amsgrad=True)

Learning rate:  0.001


In [ ]:
from tensorflow.keras import backend as K

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(350, return_sequences=True, activation='relu'), input_shape=(1, X_train.shape[2])))
model.add(Bidirectional(LSTM(350, return_sequences=True, activation='relu')))
model.add(Dense(1))
model.compile(loss="log_cosh", optimizer=adam, metrics=['mae'])

In [ ]:
mcp_save = ModelCheckpoint('LSTM_reg_18-21rmse355.keras', save_best_only=True, monitor='val_loss', mode='auto')
earlyStopping = EarlyStopping(monitor='val_loss', patience=100, verbose=1, mode='auto')

In [ ]:
#model.compile(optimizer='adam', loss='log_cosh', metrics=['accuracy'])
model.fit(X_train,y_train, epochs=5000, batch_size=32, validation_data=(X_test,y_test), callbacks=[mcp_save,earlyStopping])

Epoch 1/5000
28/28 [==============================] - 11s 126ms/step - loss: 30973.9219 - mae: 30974.6133 - val_loss: 20485.3223 - val_mae: 20486.0156
Epoch 2/5000
28/28 [==============================] - 2s 90ms/step - loss: 30932.1855 - mae: 30932.8809 - val_loss: 20409.2793 - val_mae: 20409.9707
Epoch 3/5000
28/28 [==============================] - 4s 135ms/step - loss: 30619.8770 - mae: 30620.5664 - val_loss: 20037.5391 - val_mae: 20038.2324
Epoch 4/5000
28/28 [==============================] - 3s 100ms/step - loss: 29564.1992 - mae: 29564.8887 - val_loss: 19018.1719 - val_mae: 19018.8652
Epoch 5/5000
28/28 [==============================] - 3s 96ms/step - loss: 26869.1562 - mae: 26869.8477 - val_loss: 16705.0312 - val_mae: 16705.7266
Epoch 6/5000
28/28 [==============================] - 3s 90ms/step - loss: 21267.6973 - mae: 21268.3945 - val_loss: 12298.4492 - val_mae: 12299.1406
Epoch 7/5000
28/28 [==============================] - 3s 116ms/step - loss: 13870.7959 - mae: 13871.48

KeyboardInterrupt: 

# **Testing Model**

In [ ]:
#model.save('LSTM_complete_model344.h5', overwrite=True)

In [ ]:
#from tensorflow.keras.models import load_model
#loaded_model = load_model('LSTM_complete_model344.h5')

In [ ]:
y_pred = np.ravel(model.predict(X_test))

7/7 [==============================] - 0s 13ms/step


In [ ]:
y_test=np.ravel(y_test)

In [ ]:
y_pred

array([28793.03 , 28731.791, 28987.502, 30382.396, 31583.576, 30846.406,
       29611.984, 29429.559, 29031.906, 29351.752, 30841.42 , 29550.42 ,
       30072.842, 29750.672, 29310.363, 28518.324, 27246.041, 23996.428,
       21849.098, 21135.45 , 21253.896, 20572.574, 19363.373, 19458.959,
       20441.332, 21185.264, 20520.516, 20715.145, 21263.32 , 21486.26 ,
       21551.932, 21194.648, 20917.795, 20327.059, 19505.918, 19645.754,
       19389.666, 19376.174, 19665.842, 20175.354, 20323.492, 20892.354,
       21841.102, 21710.38 , 21205.525, 20489.354, 19692.77 , 19587.941,
       20108.604, 20769.666, 20901.781, 21270.744, 21806.295, 22464.121,
       23496.984, 22909.096, 23072.121, 22392.512, 22549.674, 21804.088,
       20948.318, 21476.646, 23197.777, 23839.008, 23959.73 , 23530.416,
       22992.264, 22684.713, 22875.166, 22614.898, 22851.113, 22963.967,
       22876.066, 23725.756, 23377.016, 23397.549, 24294.045, 23889.197,
       24389.54 , 24296.607, 24140.594, 23788.613, 

In [ ]:
r2=r2_score(y_test,y_pred) #testing score/ r^2
r2

0.9733521881853215

In [ ]:
mae=mean_absolute_error(y_test,y_pred) #mae
mae

374.2749

In [ ]:
rmse=np.sqrt(mean_squared_error(y_test,y_pred)) #rmse
rmse

558.83093

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape=mean_absolute_percentage_error(y_test,y_pred) #mape
mape

1.7625730484724045

In [ ]:
pd.DataFrame(zip(['MAE','RMSE','R^2'],[mae,rmse,r2])).transpose()

,0,1,2
0,MAE,RMSE,R^2
1,374.274902,558.830933,0.973352


In [ ]:
pd.DataFrame([y_test,y_pred]).transpose()

,0,1
0,28888.0,28793.029297
1,29144.0,28731.791016
2,30589.0,28987.501953
3,31711.0,30382.396484
4,31096.0,31583.576172
...,...,...
215,16591.0,16708.806641
216,16550.0,16655.789062
217,16570.0,16608.771484
218,16559.0,16642.753906
